In [12]:
_call_price(S, K, T, r, sigma):
    """Calculate Black-Scholes call option price."""
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(S - K, 0)
    try:
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        return call_price if not (np.isnan(call_price) or call_price < 0) else max(S - K, 0)
    except (ValueError, OverflowError, ZeroDivisionError):
        return max(S - K, 0)

def black_scholes_put_price(S, K, T, r, sigma):
    """Calculate Black-Scholes put option price."""
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(K - S, 0)
    try:
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return put_price if not (np.isnan(put_price) or put_price < 0) else max(K - S, 0)
    except (ValueError, OverflowError, ZeroDivisionError):
        return max(K - S, 0)

def current_stock_price(ticker):
    """Fetches the current stock price."""
    url = f"https://eodhd.com/api/real-time/{ticker}.US?api_token={API_KEY}&fmt=json"
    print(f"--- Fetching current price for {ticker} ---")
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        price = data.get("close")
        return float(price) if price is not None else 0
    except Exception as e:
        logging.warning(f"Failed to fetch current price for {ticker}: {e}")
        return 0

# --- Step 4: Main Watchlist Analysis Functions ---
def simulate_option_watchlist_single(ticker, option_type, strike, expiration, days_to_gain,
                                     number_of_contracts, average_cost_per_contract, risk_free_rate=0.05):
    """
    Simulates option scenarios for a single watchlist contract, including Greeks and liquidity data.
    """
    try:
        # --- Data Type Conversion and Default Handling ---
        try: strike = float(strike)
        except (TypeError, ValueError): return pd.DataFrame([{"Error": f"Invalid strike value: {strike}"}])

        current_price = current_stock_price(ticker)
        if current_price <= 0: return pd.DataFrame([{"Error": f"Could not fetch a valid stock price for {ticker}."}])

        params = {
            "api_token": API_KEY, "filter[underlying_symbol]": ticker, "filter[type]": option_type,
            "filter[exp_date_from]": expiration, "filter[exp_date_to]": expiration,
            "filter[strike_from]": strike, "filter[strike_to]": strike, "page[limit]": 1
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json().get("data", [])

        if not data or "attributes" not in data[0]: return pd.DataFrame([{"Error": f"No contract found for {ticker} with specified details."}])
        
        attr = data[0]["attributes"]
        last_price = float(attr.get("last", 0))
        iv = float(attr.get("volatility", 0.3))
        if iv <= 0: iv = 0.3

        # Extract Greeks and Liquidity Data
        delta = attr.get("delta"); theta = attr.get("theta"); gamma = attr.get("gamma"); vega = attr.get("vega"); rho = attr.get("rho")
        bid = attr.get("bid"); ask = attr.get("ask"); volume = attr.get("volume"); open_interest = attr.get("open_interest", "NA")

        exp_date = datetime.strptime(expiration, "%Y-%m-%d").replace(tzinfo=timezone.utc)
        today = datetime.now(timezone.utc)

        try: days_to_gain = int(days_to_gain)
        except (TypeError, ValueError): days_to_gain = max(1, int((exp_date - today).days * 0.5))

        try: number_of_contracts = int(number_of_contracts)
        except (TypeError, ValueError): number_of_contracts = 1
            
        try:
            average_cost_per_contract = float(average_cost_per_contract)
            if average_cost_per_contract <= 0: average_cost_per_contract = last_price
        except (TypeError, ValueError): average_cost_per_contract = last_price

        eval_date = today + timedelta(days=days_to_gain)
        T_eval = max((exp_date - eval_date).days / 365, 0.0001)
        total_cost = average_cost_per_contract * number_of_contracts * 100

        # --- Scenario Simulation ---
        scenarios = [0.05, 0.10, 0.20, 0.50, 1.0, 2.0] 
        rows = []

        for pct in scenarios:
            stock_up, stock_down = current_price * (1 + pct), current_price * (1 - pct)
            if option_type == "call":
                premium_up = black_scholes_call_price(stock_up, strike, T_eval, risk_free_rate, iv)
                premium_down = black_scholes_call_price(stock_down, strike, T_eval, risk_free_rate, iv)
            else: # put
                premium_up = black_scholes_put_price(stock_up, strike, T_eval, risk_free_rate, iv)
                premium_down = black_scholes_put_price(stock_down, strike, T_eval, risk_free_rate, iv)
            
            # Calculate Total Future Value for Equity(+/-) columns
            equity_up = premium_up * number_of_contracts * 100
            equity_down = premium_down * number_of_contracts * 100
            
            if last_price > 0:
                premium_up_pct_change = ((premium_up - last_price) / last_price) * 100
                premium_down_pct_change = ((premium_down - last_price) / last_price) * 100
            else:
                premium_up_pct_change = 0
                premium_down_pct_change = 0

            rows.append({
                "Ticker": ticker, "Option Type": option_type, "Strike": strike, "Expiration": expiration,
                "Days to Gain": days_to_gain,
                "Underlying Scenario % Change": f"±{int(pct * 100)}%",
                "Current Underlying": round(current_price, 2), "Simulated Underlying (+)": round(stock_up, 2), 
                "Simulated Underlying (-)": round(stock_down, 2), "Current Premium": round(last_price, 2),
                "Simulated Premium (+)": round(premium_up, 2),
                "Simulated Premium (+) % Change": round(premium_up_pct_change, 2),
                "Simulated Premium (-)": round(premium_down, 2),
                "Simulated Premium (-) % Change": round(premium_down_pct_change, 2),
                "Number of Contracts": number_of_contracts, "Average Cost per Contract": round(average_cost_per_contract, 2),
                "Equity Invested": round(total_cost, 2),
                "Simulated Equity (+)": round(equity_up, 2), # ### RENAMED ###
                "Simulated Equity (-)": round(equity_down, 2), # ### RENAMED ###
                "Bid": bid, "Ask": ask, "Volume": volume, "Open Interest": open_interest, 
                "Implied Volatility": round(iv * 100, 2), "Delta": round(delta, 4) if delta is not None else "NA",
                "Theta": round(theta, 4) if theta is not None else "NA", "Gamma": round(gamma, 4) if gamma is not None else "NA",
                "Vega": round(vega, 4) if vega is not None else "NA", "Rho": round(rho, 4) if rho is not None else "NA",
            })

        # --- Reorder Columns for Final Output ---
        df = pd.DataFrame(rows)
        
        # Apply the new user-defined column order
        column_order = [
            "Ticker", "Option Type", "Strike", "Expiration", 
            "Days to Gain",
            "Underlying Scenario % Change", "Current Underlying", "Simulated Underlying (+)", 
            "Simulated Underlying (-)", "Current Premium", "Simulated Premium (+)", 
            "Simulated Premium (+) % Change", "Simulated Premium (-)", "Simulated Premium (-) % Change", 
            "Number of Contracts", "Average Cost per Contract", "Equity Invested", 
            "Simulated Equity (+)", "Simulated Equity (-)", # ### RENAMED ###
            "Bid", "Ask", "Volume", "Open Interest", "Implied Volatility", 
            "Delta", "Theta", "Gamma", "Vega", "Rho"
        ]
        
        if not df.empty:
            existing_columns_in_order = [col for col in column_order if col in df.columns]
            return df[existing_columns_in_order]
        return df

    except Exception as e:
        print(f"🚨 An error occurred in simulate_option_watchlist_single for {ticker}. Reason: {e}")
        return pd.DataFrame([{"Error": str(e)}])

def whole_watchlist(contract_list):
    """Processes a list of contracts and concatenates the results."""
    all_rows = []
    for contract in contract_list:
        print(f"🧪 Simulating contract: {contract}")
        contract.setdefault('days_to_gain', None)
        df = simulate_option_watchlist_single(**contract)
        if not df.empty and "Error" not in df.columns:
            print(f"✅ Successful simulation for {contract['ticker']}")
            all_rows.append(df)
        else:
            error_msg = df.iloc[0]['Error'] if not df.empty and "Error" in df.columns else "No data returned."
            print(f"❌ Simulation error for {contract['ticker']}: {error_msg}")

    return pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()


# --- Step 5: Example Usage for Jupyter Notebook ---
my_contracts_to_test = [
    {
        "ticker": "AAPL", "option_type": "call", "strike": 220,
        "expiration": "2025-09-19", "number_of_contracts": 2,
        "average_cost_per_contract": 15.50, "days_to_gain": 30
    },
    {
        "ticker": "MSFT", "option_type": "put", "strike": 450,
        "expiration": "2025-10-17", "number_of_contracts": 5,
        "average_cost_per_contract": 20.10, "days_to_gain": 45
    },
    {
        "ticker": "GOOG", "option_type": "call", "strike": 180,
        "expiration": "2025-09-19", "number_of_contracts": 10,
        "average_cost_per_contract": 10.00, "days_to_gain": 20
    }
]

# Run the analysis
watchlist_df = whole_watchlist(my_contracts_to_test)

# Display the resulting DataFrame
print("\n--- Final Watchlist Analysis Table ---")
watchlist_df


🧪 Simulating contract: {'ticker': 'AAPL', 'option_type': 'call', 'strike': 220, 'expiration': '2025-09-19', 'number_of_contracts': 2, 'average_cost_per_contract': 15.5, 'days_to_gain': 30}
--- Fetching current price for AAPL ---
✅ Successful simulation for AAPL
🧪 Simulating contract: {'ticker': 'MSFT', 'option_type': 'put', 'strike': 450, 'expiration': '2025-10-17', 'number_of_contracts': 5, 'average_cost_per_contract': 20.1, 'days_to_gain': 45}
--- Fetching current price for MSFT ---
✅ Successful simulation for MSFT
🧪 Simulating contract: {'ticker': 'GOOG', 'option_type': 'call', 'strike': 180, 'expiration': '2025-09-19', 'number_of_contracts': 10, 'average_cost_per_contract': 10.0, 'days_to_gain': 20}
--- Fetching current price for GOOG ---
✅ Successful simulation for GOOG

--- Final Watchlist Analysis Table ---


,Ticker,Option Type,Strike,Expiration,Days to Gain,Underlying Scenario % Change,Current Underlying,Simulated Underlying (+),Simulated Underlying (-),Current Premium,Simulated Premium (+),Simulated Premium (+) % Change,Simulated Premium (-),Simulated Premium (-) % Change,Number of Contracts,Average Cost per Contract,Equity Invested,Simulated Equity (+),Simulated Equity (-),Bid,Ask,Volume,Open Interest,Implied Volatility,Delta,Theta,Gamma,Vega,Rho
0,AAPL,call,220.0,2025-09-19,30,±5%,213.92,224.62,203.23,7.15,9.40,31.52,1.12,-84.32,2,15.5,3100.0,1880.77,224.26,7.00,7.20,4371,26020,26.26,0.4470,-0.0851,0.0175,0.3406,0.1434
1,AAPL,call,220.0,2025-09-19,30,±10%,213.92,235.31,192.53,7.15,17.49,144.65,0.20,-97.19,2,15.5,3100.0,3498.56,40.25,7.00,7.20,4371,26020,26.26,0.4470,-0.0851,0.0175,0.3406,0.1434
2,AAPL,call,220.0,2025-09-19,30,±20%,213.92,256.71,171.14,7.15,37.60,425.85,0.00,-99.99,2,15.5,3100.0,7519.70,0.19,7.00,7.20,4371,26020,26.26,0.4470,-0.0851,0.0175,0.3406,0.1434
3,AAPL,call,220.0,2025-09-19,30,±50%,213.92,320.88,106.96,7.15,101.70,1322.31,0.00,-100.00,2,15.5,3100.0,20339.04,0.00,7.00,7.20,4371,26020,26.26,0.4470,-0.0851,0.0175,0.3406,0.1434
4,AAPL,call,220.0,2025-09-19,30,±100%,213.92,427.84,0.00,7.15,208.66,2818.27,0.00,-100.00,2,15.5,3100.0,41731.24,0.00,7.00,7.20,4371,26020,26.26,0.4470,-0.0851,0.0175,0.3406,0.1434
5,AAPL,call,220.0,2025-09-19,30,±200%,213.92,641.77,-213.92,7.15,422.58,5810.18,0.00,-100.00,2,15.5,3100.0,84515.64,0.00,7.00,7.20,4371,26020,26.26,0.4470,-0.0851,0.0175,0.3406,0.1434
6,MSFT,put,450.0,2025-10-17,45,±5%,504.49,529.71,479.26,5.25,0.40,-92.42,4.83,-7.98,5,20.1,10050.0,199.09,2415.47,5.15,5.75,46,1745,26.01,-0.1497,-0.0783,0.0036,0.5738,-0.1732
7,MSFT,put,450.0,2025-10-17,45,±10%,504.49,554.93,454.04,5.25,0.09,-98.35,12.45,137.11,5,20.1,10050.0,43.25,6224.22,5.15,5.75,46,1745,26.01,-0.1497,-0.0783,0.0036,0.5738,-0.1732
8,MSFT,put,450.0,2025-10-17,45,±20%,504.49,605.38,403.59,5.25,0.00,-99.95,46.00,776.24,5,20.1,10050.0,1.27,23001.17,5.15,5.75,46,1745,26.01,-0.1497,-0.0783,0.0036,0.5738,-0.1732
9,MSFT,put,450.0,2025-10-17,45,±50%,504.49,756.73,252.24,5.25,0.00,-100.00,195.30,3619.97,5,20.1,10050.0,0.00,97649.24,5.15,5.75,46,1745,26.01,-0.1497,-0.0783,0.0036,0.5738,-0.1732


In [11]:
watchlist_df.to_csv('watchlistTest.csv')